# Filter background from D&F images.
We have combined DAPI (blue) and FLUORO (green) images.  
Partition each image into patches.  
Try to select the mostly blue patches.

In [1]:
import os
import glob
import cv2 # OpenCV-Python
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
DIR_IMAGES_RAW = "/home/jrm/Martinez/images/raw/"
PATTERN_IMAGES_RAW = "*.DF1.*.tif"
PATCH_SIZE=224  # matches VGG

In [2]:
def get_image_names(path,pattern):
    #RAW_IMAGE_NAMES = os.listdir(DIR_IMAGES_RAW)
    paths = glob.glob(path+pattern)
    names = [os.path.basename(x) for x in paths]
    return names
FILENAMES_IMAGES_RAW = get_image_names(DIR_IMAGES_RAW,PATTERN_IMAGES_RAW)
FILENAMES_IMAGES_RAW

['F15.DF1.135.tif',
 'D5.DF1.43.tif',
 'H3.DF1.27.tif',
 'C11.DF1.96.tif',
 'H13.DF1.114.tif',
 'I1.DF1.09.tif',
 'G3.DF1.25.tif',
 'C1.DF1.01.tif',
 'F11.DF1.94.tif',
 'F9.DF1.80.tif',
 'H7.DF1.62.tif',
 'A3.DF1.26.tif',
 'A5.DF1.45.tif',
 'H15.DF1.132.tif',
 'D1.DF1.02.tif',
 'G15.DF1.133.tif',
 'B15.DF1.134.tif',
 'I5.DF1.48.tif',
 'E9.DF1.75.tif',
 'I13.DF1.117.tif',
 'F3.DF1.22.tif',
 'B7.DF1.60.tif',
 'B13.DF1.115.tif',
 'E7.DF1.64.tif',
 'F7.DF1.63.tif',
 'D3.DF1.20.tif',
 'E5.DF1.42.tif',
 'H1.DF1.07.tif']

In [3]:
def load_pixel_array(path,filename,verbose=False):
    im = Image.open(path+filename)
    ima = np.array(im)   # convert to numpy
    if verbose:
        print(filename, ima.size, ima.shape)
    return ima
pixel_array = load_pixel_array(DIR_IMAGES_RAW,'H1.DF1.07.tif',True)

H1.DF1.07.tif 479598264 (12513, 12776, 3)


In [4]:
avg = np.average(pixel_array,axis=(0,1))
std = np.std(pixel_array,axis=(0,1))
print("Average",avg,"STD",std)
print("Example pixel:",pixel_array[0,0])

Average [ 0.         40.01907205 28.02747956] STD [ 0.         25.25630313 43.83137465]
Example pixel: [ 0 35  2]


In [5]:
PATCH_THRESHOLD=PATCH_SIZE*PATCH_SIZE/2
IM_WIDTH = pixel_array.shape[0]
IM_HEIGHT = pixel_array.shape[1]

In [25]:
W=0
H=0
def get_next_patch(pixels): # assume square
    global W,H
    if W+PATCH_SIZE>IM_WIDTH:
        H += PATCH_SIZE
        W = 0
    if H+PATCH_SIZE>IM_HEIGHT:
        patch = None
    if W+PATCH_SIZE<=IM_WIDTH and H+PATCH_SIZE<=IM_HEIGHT:
        patch = pixels[W:W+PATCH_SIZE, H:H+PATCH_SIZE]
        W += PATCH_SIZE
    return patch

In [26]:
BACKGROUND_MAX=np.array( [20,20,20] )
GREEN_MIN=np.array( [0,40,0] )
BLUE_MIN=np.array( [0,0,40] )
num_back_patches = 0
num_green_patches = 0
num_blue_patches = 0
patch = get_next_patch(pixel_array)
while patch is not None:
    num_background_pixels = np.sum(np.all(patch<BACKGROUND_MAX,axis=2))
    if num_background_pixels>=PATCH_THRESHOLD:
        num_back_patches += 1
    else:
        num_green_pixels = np.sum(np.all(patch>=GREEN_MIN,axis=2))
        if num_green_pixels>=PATCH_THRESHOLD:
            num_green_patches += 1
        else:
            num_blue_pixels = np.sum(np.all(patch>=BLUE_MIN,axis=2))
            if num_blue_pixels>=PATCH_THRESHOLD:
                num_blue_patches += 1
                pic = Image.fromarray(patch)
                name = "BLUE{}.tif".format(num_blue_patches)
                pic.save(name)
    patch = get_next_patch(pixel_array)
print("back=",num_back_patches,"green=",num_green_patches,"blue=",num_blue_patches)

back= 356 green= 1190 blue= 3


The three blue images did indeed have solid blue nuclei.  
Surprisingly, they also had slight green smear background.